E06: meta-exercise! Think of a fun/interesting exercise and complete it.

My idea is to use the model on a different data set, and see how it performs. I will use German street names, from the [OpenAdresses dataset of Germany](https://www.kaggle.com/datasets/openaddresses/openaddresses-europe?resource=download&select=germany.csv)

It is advised to run this notebook on a GPU, as it will take a long time to train on a CPU. I ran it on a Google Colab GPU.

In [248]:
import torch
import torch.nn.functional as F
from math import floor

In [249]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [250]:
device

device(type='cuda')

In [251]:
# Read in the dataset
# Adjust the path to the file on your computer or in the cloud
streetnames = open('./street-names.txt', 'r', encoding='utf-8').read().splitlines()

In [252]:
streetnames[:10]

['am hohen rand',
 'lehrer-geßner-straße',
 'wittekindallee',
 'am darloh',
 'taxusweg',
 'hinterm friedhof',
 'kleine wende',
 'an der liff',
 'posthorn',
 'landsberger allee']

In [253]:
# Create training, dev, and test sets
train_index = floor(len(streetnames) * 0.90)
dev_index = floor(len(streetnames) * 0.95)

train = streetnames[:train_index]
dev = streetnames[train_index:dev_index]
test = streetnames[dev_index:]

In [254]:
special_token = '.'

In [255]:
chars = sorted(list(set(''.join([special_token] + streetnames))))

# Create look up tables for the alphabet and other characters.
  # stoi = string to index
  # itos = index to string
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

In [256]:
itos.values()

dict_values([' ', '&', "'", '-', '.', ':', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ß', 'ä', 'ö', 'ü'])

In [257]:
chars_size = len(chars)
chars_size

36

In [258]:
xs_train,  ys_train = [], []

# Create the training data
# input xs: (ch1, ch2) 
# prediction ys: ch3
for streetname in train:

  # prepend two special characters and append one special characters to each streetname
  chs = [special_token] * 2 + list(streetname) + [special_token]
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs_train.append((ix1, ix2))
    ys_train.append(ix3)

num = len(xs_train)
print('number of training examples: ', num)
xs_train = torch.tensor(xs_train, device=device)
ys_train = torch.tensor(ys_train, device=device)

number of training examples:  1965683


In [259]:
# Create the dev data
xs_dev,  ys_dev = [], []
for word in dev:
  chs = [special_token] * 2 + list(word) + [special_token]
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs_dev.append((ix1, ix2))
    ys_dev.append(ix3)

xs_dev = torch.tensor(xs_dev, device=device)
ys_dev = torch.tensor(ys_dev, device=device)
print('number of development examples: ', xs_dev.nelement())

number of development examples:  40084


In [260]:
# Create the test data
xs_test,  ys_test = [], []
for word in test:
  chs = [special_token] * 2 + list(word) + [special_token]
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs_test.append((ix1, ix2))
    ys_test.append(ix3)

xs_test = torch.tensor(xs_test, device=device)
ys_test = torch.tensor(ys_test, device=device)
print('number of testing examples: ', xs_test.nelement())

number of testing examples:  39736


In [261]:
g = torch.Generator(device=device).manual_seed(2147483647)
W = torch.randn((chars_size*2, chars_size), generator=g, device=device, requires_grad=True)

In [262]:
# gradient descent
iterations = 500
learning_rate = 200
smoothing_strenth = 0.01

for k in range(iterations):

  # forward pass
  xenc= F.one_hot(xs_train, num_classes=chars_size).float()
  xenc_flat = xenc.flatten(1) # flatten the one-hot encoded input vector
  logits = xenc_flat @ W # predict log-counts
  loss = F.cross_entropy(logits, ys_train) + smoothing_strenth * (W**2).mean() # compute loss
  # print loss every 10% of iterations
  if k % floor(iterations/10) == 0:
    print(f'loss at step {k}: {loss.item():.3f}')

  # backward pass
  W.grad = None # flush the gradients
  loss.backward()

  # update step
  cooldown = 1.0 / (1 + 0.001 * k)
  W.data += -learning_rate * cooldown * W.grad
print(f'final training loss: {loss.item():.3f}, with smoothing strength {smoothing_strenth}')

loss at step 0: 4.512
loss at step 10: 2.472
loss at step 20: 2.283
loss at step 30: 2.208
loss at step 40: 2.169
loss at step 50: 2.145
loss at step 60: 2.128
loss at step 70: 2.117
loss at step 80: 2.109
loss at step 90: 2.102
final training loss: 2.097, with smoothing strength 0.01


In [263]:
# Evaluate the model on the dev set
xenc= F.one_hot(xs_dev, num_classes=chars_size).float()
xenc_flat = xenc.flatten(1) # flatten the one-hot encoded input vector
logits = xenc_flat @ W # predict log-counts
# softmax
counts = logits.exp() # counts
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# loss function (cross-entropy) + regularization (L2)
loss = F.cross_entropy(logits, ys_dev)
print(f'loss on dev set: {loss.item():.3f}, with smoothing strength {smoothing_strenth}')

loss on dev set: 2.089, with smoothing strength 0.01


In [264]:
# Evaluate the model on the test set
xenc= F.one_hot(xs_test, num_classes=chars_size).float()
xenc_flat = xenc.flatten(1) # flatten the one-hot encoded input vector
logits = xenc_flat @ W # predict log-counts
# softmax
counts = logits.exp() # counts
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# loss function (cross-entropy) + regularization (L2)
loss = F.cross_entropy(logits, ys_test) + smoothing_strenth * (W**2).mean() # compute loss
print(f'loss on test set: {loss.item():.3f}')

loss on test set: 2.109


In [266]:
g = torch.Generator(device=device).manual_seed(2147483647)
# sample street names
name_count = 100
sampled_street_names = []
for i in range(name_count):

  out = []
  ix1 = stoi[special_token]
  ix2 = stoi[special_token]

  while True:
    xenc = F.one_hot(torch.tensor([ix1, ix2], device=device), num_classes=chars_size).float()
    xenc_flat = xenc.flatten()
    logits = xenc_flat @ W # predict log-counts
    # softmax
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(0, keepdims=True) # probabilities for next character
    
    # move index to next character
    ix1 = ix2
    ix2 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix2])
    if ix2 == stoi[special_token]:
      # stop if we reach the end of the word
      # 0 is the index of the special character '.'
      break

  sampled_street_names.append(''.join(out))
for streetname in sampled_street_names:
  print(streetname)

ga'ntistraße.
qmelb.
hanwkh.
ber kachrebskend.
aner straße.
bah-z-weg.
aufsweg.
dimp.
rindruck.
haldimpek.
em-nel.
sendenüstraßekfedhep.
eueg.
imgswim ben-ung-stjö dek mperhäeweg.
zur sanam wickldergen de.
ausweg.
mmnben ste.
her-aße.
nnade.
witraße.
haundeit.
litran.
äme-d.
weg.
kornstr der wegern.
traße.
bustraßqscheytraße.
bengeneg.
ploweg.
aun-ld.
strasteraße.
mm.
raße.
lashöuqgastritzsthng.
rintrtraße.
der brstristraße.
gr-stjaltraße.
karoße.
aidek.
ooskühle.
gaughof.
p:ang.
alem-hofeiwel.
vmperlt.
mablen.
brobelteltselskrit.
erger straße.
düttel.
ll' st.
kimeler derteöllitz-wer ok.
vmersiehruen.
kor-traße.
er we-mbickurnschatzeiderg.
margße.
pastraße.
bucherstraße.
ptraße.
kllannuck.
almeödskrhof-wied.
anshaldedengen.
erg.
alttam -raße.
allustenstrochgsxüen-saße.
dor hof dorerm weg.
me.
bokiegh.
orheim duer bergender dorsthornchaweg.
eipfä.
för bick.
zlickller steeberzumratze&.
tzlstramz.
der strackserweg straße.
raße.
ben dg.
dostz.
scherg.
ome.
cherenndimpeieb&plamplaße.
matzäl